In [152]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.datasets import load_svmlight_file
from datetime import timedelta
import re

## Overview
The baseline model is trained using the following features:

1) Diagnosis (value 1 for every diagnosis)

2) Lab_results (value is the mean of recorded values, scaled by maximum

In [153]:
### Sampling on patients
path = "../../data/"
patients = pd.read_csv(f"{path}PATIENTS.csv")
#ensuring every patient is unique
print(f"{len(patients.SUBJECT_ID.unique())} unique patients in {len(patients)} rows")
#sampling random patients
patients_sample = patients.sample(n = 10000, random_state= 1)


46520 unique patients in 46520 rows


In [154]:
# get training and test indices
from sklearn.model_selection import train_test_split
train_indices, test_indices = train_test_split(patients_sample['SUBJECT_ID'].unique(), test_size = 0.2, random_state = 1)

In [155]:
#get admissions
admissions = pd.read_csv(f"{path}ADMISSIONS.csv.gz")
admissions = pd.merge(patients_sample.SUBJECT_ID,admissions)
#admissions

In [156]:
# Global Constants
OBSERVATION_WINDOW = 2000
PREDICTION_WINDOW = 50
USEFUL_COLUMNS = ['SUBJECT_ID','FEATURE','DATE','VALUE']

In [157]:
patients_sample

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
2811,3850,4074,M,2121-08-01 00:00:00,2204-02-06 00:00:00,2204-02-06 00:00:00,2204-02-06 00:00:00,1
36909,44143,90889,M,2095-04-29 00:00:00,NaN,NaN,NaN,0
42305,39453,72753,M,2042-06-01 00:00:00,NaN,NaN,NaN,0
39630,37377,64908,M,2108-04-22 00:00:00,NaN,NaN,NaN,0
46355,38824,70273,F,2090-05-28 00:00:00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
18708,20369,21583,F,2105-06-22 00:00:00,2189-06-08 00:00:00,2189-06-08 00:00:00,2189-06-08 00:00:00,1
40544,43127,87048,M,2045-05-20 00:00:00,NaN,NaN,NaN,0
20762,15860,16761,F,2065-05-04 00:00:00,NaN,NaN,NaN,0
22080,27037,28702,F,2107-07-07 00:00:00,2162-04-14 00:00:00,NaN,2162-04-14 00:00:00,1


In [158]:
# note_events = pd.read_csv("../data/NOTEEVENTS.csv")
# #sample and save for these 1000 patients
# note_events_sample = pd.merge(patients_sample.SUBJECT_ID,note_events)
# note_events_sample.to_csv("../data/NOTEEVENTS_SAMPLE.csv")

In [159]:
### Incomplete: Using AWS glue, for now I will do stuff locally
# import boto3
# Creating the low level functional client
# client = boto3.client(
#     'glue',
#     aws_access_key_id = '',
#     aws_secret_access_key = '',
#     region_name = 'us-east-1'
# )
# clientResponse = client.get_table(DatabaseName="mimiciii",Name="admissions")

In [160]:
def convert_icd9(icd9_object):
    """
    :param icd9_object: ICD-9 code (Pandas/Numpy object).
    :return: extracted main digits of ICD-9 code
    """
    icd9_str = str(icd9_object)

    if icd9_str[0] == 'E': #if code starts with E
        converted = icd9_str[:4]
    else: #if they start with V or numeric
        converted = icd9_str[:3]

    return converted

def build_codemap(dataset):
    """
    :return: Dict of code map {main-digits of ICD9: unique feature ID}
    """
    # TODO: We build a code map using ONLY train data. Think about how to construct validation/test sets using this.
    df_digits = dataset['FEATURE'].unique()
    codemap = {}
    for i in range(0,len(df_digits)):
        codemap[df_digits[i]] = i


    return codemap

# #dataset that contains HADM_ID
# def filter_by_time_window(dataset,admissions,patients):
#     dataset = pd.merge(dataset,admissions[['HADM_ID','ADMITTIME']],on = ['HADM_ID'])
#     dataset = pd.merge(dataset,patients[['SUBJECT_ID','DOD']])
#     dataset.loc[]
    

### Adding diagnoses data

In [161]:
#read and etl on diagnoses
diagnoses = pd.read_csv(f"{path}DIAGNOSES_ICD.csv.gz")
# sample for the patients
diagnoses = pd.merge(patients_sample.SUBJECT_ID,diagnoses)

#add admit time to diagnosis 
diagnoses = diagnoses.merge(admissions[['HADM_ID','ADMITTIME']],on = 'HADM_ID')

#convert features
diagnoses['VALUE'] = 1
diagnoses["ICD9_CODE"] = diagnoses["ICD9_CODE"].apply(convert_icd9).apply(lambda x: f"DIAG_{x}")
diagnoses = diagnoses.rename(columns = {'ICD9_CODE':'FEATURE',"ADMITTIME":'DATE'})
diagnoses = diagnoses[USEFUL_COLUMNS]

diagnoses.drop_duplicates(inplace = True)


In [162]:
diagnoses

,SUBJECT_ID,FEATURE,DATE,VALUE
0,4074,DIAG_038,2204-02-04 07:26:00,1
1,4074,DIAG_785,2204-02-04 07:26:00,1
2,4074,DIAG_578,2204-02-04 07:26:00,1
3,4074,DIAG_427,2204-02-04 07:26:00,1
4,4074,DIAG_428,2204-02-04 07:26:00,1
...,...,...,...,...
139065,62212,DIAG_790,2176-06-09 13:01:00,1
139066,62212,DIAG_781,2176-06-09 13:01:00,1
139067,62212,DIAG_338,2176-06-09 13:01:00,1
139068,62212,DIAG_V15,2176-06-09 13:01:00,1


### Lab Results

In [163]:

#read and etl on lab_results
# lab_results = pd.read_csv(f"{path}LABEVENTS_SAMPLE.csv")
lab_results = pd.read_csv(f"{path}LABEVENTS.csv.gz")
lab_results = pd.merge(patients_sample.SUBJECT_ID,lab_results)
# lab_results.to_csv("LABEVENTS_SAMPLE.csv")


In [164]:
#roughly 20% of these items have null in the HADM ID
lab_results = lab_results.rename(columns = {'CHARTTIME':'DATE','ITEMID':'FEATURE'})
#making sure lab_results has different item_id than diagnostics. appending a LAB to the itemIDs
lab_results['FEATURE'] = lab_results['FEATURE'].apply(lambda x: f"LAB_{x}")
lab_results = lab_results[USEFUL_COLUMNS]
lab_results

,SUBJECT_ID,FEATURE,DATE,VALUE
0,4074,LAB_51279,2203-10-16 07:30:00,3.19
1,4074,LAB_51301,2203-10-16 07:30:00,8.5
2,4074,LAB_51221,2203-10-16 21:15:00,27.4
3,4074,LAB_50868,2203-10-17 05:10:00,14
4,4074,LAB_50882,2203-10-17 05:10:00,18
...,...,...,...,...
5882837,62212,LAB_51250,2176-06-12 09:50:00,88
5882838,62212,LAB_51265,2176-06-12 09:50:00,287
5882839,62212,LAB_51277,2176-06-12 09:50:00,14.5
5882840,62212,LAB_51279,2176-06-12 09:50:00,3.61


In [165]:
#take average value for lab_results
lab_results = lab_results.dropna()
#keep only numeric values, removing things like 'not done'
def is_a_number(x):
    try:
        float(x.strip())
        return True
    except:
        return False
    return True
lab_results = lab_results.loc[lab_results['VALUE'].apply(is_a_number),:]
lab_results['VALUE'] = pd.to_numeric(lab_results['VALUE'])

#TODO: filter by date

#keep average of values (can use more sophisticated methods later on)
# lab_results = lab_results.groupby(['SUBJECT_ID','FEATURE']).mean().reset_index()
lab_results

,SUBJECT_ID,FEATURE,DATE,VALUE
0,4074,LAB_51279,2203-10-16 07:30:00,3.19
1,4074,LAB_51301,2203-10-16 07:30:00,8.50
2,4074,LAB_51221,2203-10-16 21:15:00,27.40
3,4074,LAB_50868,2203-10-17 05:10:00,14.00
4,4074,LAB_50882,2203-10-17 05:10:00,18.00
...,...,...,...,...
5882837,62212,LAB_51250,2176-06-12 09:50:00,88.00
5882838,62212,LAB_51265,2176-06-12 09:50:00,287.00
5882839,62212,LAB_51277,2176-06-12 09:50:00,14.50
5882840,62212,LAB_51279,2176-06-12 09:50:00,3.61


### Microbiology Events

In [166]:
### reading in data similar to lab_results, like microbiology events
#NOTE: Does not seem to help at all
mb_events = pd.read_csv(f"{path}MICROBIOLOGYEVENTS.csv.gz")
mb_events = pd.merge(patients_sample.SUBJECT_ID,mb_events)
mb_events = mb_events[['SUBJECT_ID','AB_ITEMID','INTERPRETATION','CHARTTIME']]
mb_events.dropna(inplace= True)

#transform seperate boolean values for Resistant, intermediate and Sensitive. 
mb_events = mb_events.loc[mb_events.INTERPRETATION != 'P']

#keep only the last index in mb_events
mb_events.drop_duplicates(subset = ['SUBJECT_ID','AB_ITEMID'], inplace = True,keep = 'last')

mb_events['FEATURE'] = mb_events['AB_ITEMID'].apply(lambda x: f"MB_{int(x)}_") + mb_events.INTERPRETATION
mb_events.drop('AB_ITEMID',axis = 1,inplace = True)

mb_events.INTERPRETATION = 1
mb_events.rename(columns = {'CHARTTIME':'DATE','INTERPRETATION':'VALUE'},inplace = True)
mb_events = mb_events[USEFUL_COLUMNS]
display(mb_events)

,SUBJECT_ID,FEATURE,DATE,VALUE
2,4074,MB_90025_R,2200-09-24 01:00:00,1
3,4074,MB_90016_R,2200-09-24 01:00:00,1
5,4074,MB_90012_I,2200-09-24 01:00:00,1
6,4074,MB_90002_R,2200-09-24 01:00:00,1
20,4074,MB_90004_S,2204-01-16 04:44:00,1
...,...,...,...,...
133781,28702,MB_90004_S,2160-08-17 06:20:00,1
133782,28702,MB_90002_S,2160-08-17 06:20:00,1
133783,28702,MB_90015_R,2160-08-17 06:20:00,1
133784,28702,MB_90012_S,2160-08-17 06:20:00,1


### Medications

In [167]:
meds = pd.read_csv(f"{path}PRESCRIPTIONS.csv.gz")
meds = pd.merge(patients_sample.SUBJECT_ID,meds)

/Users/hassan/opt/anaconda3/envs/Homework1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [168]:
# meds = meds_original.copy()
meds = meds[meds.ENDDATE.notna()]
meds = meds[meds.GSN.notna()]

meds['DATE'] = pd.to_datetime(meds['ENDDATE'])
def find_mean_dose(dose: str) -> float:
    if pd.isnull(dose):
        return 0
    try:
        cleaned = re.sub(r'[A-Za-z,>< ]', '', dose)
        parts = cleaned.split('-')
        return np.array(parts).astype(float).mean()
    except:
        print(dose)
meds['VALUE'] = meds['DOSE_VAL_RX'].map(find_mean_dose)
meds['FEATURE'] = "MED_" + meds['GSN'].astype(str)
meds = meds[USEFUL_COLUMNS]

1.26mg/6
1.26mg/6
-15-30
-0.5-2
1%
1.25/3


In [169]:
meds

,SUBJECT_ID,FEATURE,DATE,VALUE
0,4074,MED_001972,2204-02-05,250.00
1,4074,MED_005068,2204-02-05,60.00
2,4074,MED_006549,2204-02-05,5000.00
3,4074,MED_000015,2204-02-05,0.25
4,4074,MED_001210,2204-02-05,500.00
...,...,...,...,...
886272,62212,MED_040819,2176-06-12,4.50
886273,62212,MED_046241,2176-06-13,25.00
886274,62212,MED_006549,2176-06-13,5000.00
886275,62212,MED_027413,2176-06-13,0.00


### Admissions

In [170]:
#ETL on admissions


#feature is the admission type and value is the time spent

admissions['TIME_SPENT'] = pd.to_timedelta(pd.to_datetime(admissions.DISCHTIME)-  pd.to_datetime(admissions.ADMITTIME)).dt.total_seconds()/3600
admissions['FEATURE'] = admissions.ADMISSION_TYPE.apply(lambda x: f"ADM_{x[0:4]}")
admissions.rename(columns = {'TIME_SPENT':'VALUE','ADMITTIME': 'DATE'},inplace = True)

admissions = admissions[USEFUL_COLUMNS]

# admissions = admissions.groupby(['SUBJECT_ID','FEATURE']).aggregate('sum').reset_index()


In [171]:

admissions

,SUBJECT_ID,FEATURE,DATE,VALUE
0,4074,ADM_EMER,2200-09-15 02:08:00,281.650000
1,4074,ADM_EMER,2204-01-05 10:48:00,270.533333
2,4074,ADM_EMER,2204-02-04 07:26:00,42.233333
3,90889,ADM_EMER,2153-02-05 11:21:00,348.633333
4,72753,ADM_EMER,2129-04-05 16:40:00,69.950000
...,...,...,...,...
12656,21583,ADM_EMER,2189-06-04 22:12:00,81.883333
12657,87048,ADM_ELEC,2118-09-12 14:00:00,121.500000
12658,16761,ADM_EMER,2126-05-16 22:24:00,642.600000
12659,28702,ADM_EMER,2160-07-30 19:43:00,903.950000


## Putting it all together
This step will
- merge all the features together
- give them unique feature_ids by building a codemape
- scale the features by their maximums

In [172]:
#merging diagnosis and lab_results
# lab_results.columns = ['SUBJECT_ID','FEATURE','VALUE']
# diagnoses.columns = ['SUBJECT_ID','FEATURE','VALUE']
# mb_events.columns = ['SUBJECT_ID','FEATURE','VALUE']
# admissions.columns = ['SUBJECT_ID','FEATURE','VALUE']

features = pd.concat([lab_results,diagnoses,mb_events,admissions,meds])

#get rid of missing values
features = features.dropna()

#keep only observation_window by getting index dates
features['DATE'] = pd.to_datetime(features['DATE'])
index_date = features[['SUBJECT_ID','DATE']].groupby(['SUBJECT_ID']).aggregate('max').reset_index()
index_date = index_date.merge(patients[['SUBJECT_ID','DOD']])
index_date.loc[index_date.DOD.notna(),'DATE'] = pd.to_datetime(index_date.DOD) - timedelta(days = PREDICTION_WINDOW)
index_date = index_date.drop('DOD',axis = 1).rename(columns = {'DATE':'INDEX_DATE'})
index_date['INDEX_DATE'] = pd.to_datetime(index_date['INDEX_DATE'])

#create function to process data
def process_data(features, index_date, functions: list, rarity_threshold):
    #filter on index date
    features = features.merge(index_date)
    features['DATE'] = pd.to_datetime(features['DATE'])
    features = features.loc[features.DATE<=features.INDEX_DATE] #lose observations above window
    features = features.loc[features.DATE>=features.INDEX_DATE-timedelta(days = OBSERVATION_WINDOW)] #lose observations before window
    features = features[['SUBJECT_ID','FEATURE','VALUE']]
    
    
    #keep only features that meet rarity threshold from training set
    prev_feature_count = len(features.FEATURE.unique())
    feature_count = features[features.SUBJECT_ID.isin(train_indices)][['FEATURE','VALUE']].groupby(['FEATURE']).aggregate('count').reset_index()
    acceptable_features = feature_count.loc[feature_count.VALUE > rarity_threshold, 'FEATURE']
    new_features_count = len(acceptable_features)
    features = features.loc[features.FEATURE.isin(acceptable_features),:]
    
    #aggregate on each function
    aggregations = []
    grouped = features.groupby(['SUBJECT_ID','FEATURE'])
    for i in range(len(functions)):
        agg = grouped.aggregate(functions[i]).reset_index()
        agg['FEATURE'] = agg['FEATURE'].str.cat([f'_{i}']*len(agg))
        aggregations.append(agg)
    features = pd.concat(aggregations)
    
    print(f"Kept {new_features_count} out of {prev_feature_count} features")
    
    return features


In [173]:
#filter and aggregate features
lab_results_processed = process_data(lab_results,index_date,['mean', 'max', 'min'],50)
diagnoses_processed = process_data(diagnoses,index_date,['mean','max'],30)
mb_events_processed = process_data(mb_events,index_date,['mean'],10)
admissions_processed = process_data(admissions,index_date,['sum'],0)
meds_processed = process_data(meds,index_date,['mean', 'count'],50)

features =  pd.concat([lab_results_processed,diagnoses_processed,mb_events_processed,admissions_processed,meds_processed])


Kept 250 out of 430 features
Kept 305 out of 918 features
Kept 68 out of 83 features
Kept 4 out of 4 features
Kept 753 out of 3119 features


In [174]:
features

,SUBJECT_ID,FEATURE,VALUE
0,11,LAB_50802_0,2.00
1,11,LAB_50804_0,25.00
2,11,LAB_50806_0,105.00
3,11,LAB_50808_0,1.09
4,11,LAB_50809_0,94.00
...,...,...,...
268339,99995,MED_043952_1,3.00
268340,99995,MED_045309_1,3.00
268341,99995,MED_050631_1,1.00
268342,99995,MED_051880_1,2.00


In [175]:


#scale features
max_features = features.groupby(['FEATURE']).aggregate('max').reset_index()
features = features.merge(max_features,on = 'FEATURE', suffixes=['', '_MAX'])
features['VALUE'] = features['VALUE']/features['VALUE_MAX']
features = features[['SUBJECT_ID','FEATURE','VALUE']]
features = features.dropna()
n_features = len(features.FEATURE.unique())+10
print(f"number of features: {n_features}")
display(features[0:5])

number of features: 2945


,SUBJECT_ID,FEATURE,VALUE
0,11,LAB_50802_0,0.077670
1,22,LAB_50802_0,0.000000
2,39,LAB_50802_0,-0.108738
3,71,LAB_50802_0,-0.252427
4,87,LAB_50802_0,-0.165049


In [176]:
#create mortality table
mortality = patients_sample.copy()
mortality['DEAD'] = 1- mortality['DOD_SSN'].isna()
mortality.index = mortality.SUBJECT_ID
mortality = mortality['DEAD']
display(mortality.value_counts())
display(mortality[0:5])
mortality = mortality.to_dict()

0    7190
1    2810
Name: DEAD, dtype: int64

SUBJECT_ID
4074     1
90889    0
72753    0
64908    0
70273    0
Name: DEAD, dtype: int64

### Saving the data prior to model training
The data will be saved in an SVM_light format for reproducibility

In [177]:
# split into training and test
# from sklearn.model_selection import train_test_split
# train_indices,test_indices = train_test_split(features['SUBJECT_ID'].unique(), test_size = 0.2, random_state = 1)

train = features.loc[features.SUBJECT_ID.isin(train_indices)]
test = features.loc[features.SUBJECT_ID.isin(test_indices)]

In [178]:
#apply codemap to features
codemap = build_codemap(train)

# function to help output data
def create_svmlite(patient_features, mortality, output_type):
    #apply codemap
    patient_features['FEATURE'] = patient_features['FEATURE'].map(codemap)
    patient_features['VALUE'] = patient_features.VALUE.round(6)

    
    #turn patient_features into an svm_light format kinda dictionaries
    patient_features['F2V'] = list(zip(patient_features.FEATURE,patient_features.VALUE))
    patient_features = patient_features.groupby(['SUBJECT_ID'])['F2V'].apply(list)
    patient_features = patient_features.to_dict()

    patient_ids = list(patient_features.keys())
    patient_ids.sort()
    d1 = ""
    for id in patient_ids:
        patient_features[id].sort()
        features = ''
        for feature in patient_features[id]:
            features += f" {str(int(feature[0]))}:" + "{:.6f}".format(feature[1])
        if output_type == 1: d1 += f"{mortality[id]}{features} \n"
        if output_type == 2: d1 += f"{int(id)} {mortality[id]}{features} \n"

    return d1

train_file = open(f"{path}svm_light/features.train", 'wb')
bytes_written = train_file.write(bytes((create_svmlite(train, mortality, 1)), 'UTF-8'))
print(f"wrote {bytes_written} bytes for training")
test_file = train_file = open(f"{path}svm_light/features.test", 'wb')
bytes_written = train_file.write(bytes((create_svmlite(test, mortality, 1)), 'UTF-8'))
print(f"wrote {bytes_written} bytes for test")


/Users/hassan/opt/anaconda3/envs/Homework1/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/hassan/opt/anaconda3/envs/Homework1/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/hassan/opt/anaconda3/envs/Homework1/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

wrote 21893386 bytes for training
wrote 5509683 bytes for test


### Model Training

In [179]:
# input: Name of classifier, predicted labels, actual labels
def display_metrics(classifierName, Y_pred, Y_true):
    print("______________________________________________")
    print(("Classifier: " + classifierName))
    auc_ = roc_auc_score( Y_true,Y_pred)
    print(("AUC: " + str(auc_)))
    print("______________________________________________")
    print("")

# input: X_train, Y_train and X_test
# output: Y_pred
def logistic_regression_pred(X_train, Y_train, X_test):
    log_model = LogisticRegression(random_state=1)
    log_model.fit(X_train, Y_train)
    Y_pred = log_model.predict_proba(X_test)
    return Y_pred

def random_forest_pred(X_train, Y_train, X_test):
    model = RandomForestClassifier(random_state=1)
    model.fit(X_train, Y_train)
    Y_pred = model.predict_proba(X_test)[:,1]
    return Y_pred

X_train, Y_train = load_svmlight_file(f"{path}svm_light/features.train", n_features=n_features)
X_test, Y_test = load_svmlight_file(f"{path}svm_light/features.test", n_features=n_features)


### Results

In [180]:
from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
# display_metrics("Logistic Regression", logistic_regression_pred(X_train, Y_train, X_test), Y_test)
# display_metrics("SVM", svm_pred(X_train, Y_train, X_test), Y_test)
display_metrics("Random Forest Tree", random_forest_pred(X_train, Y_train, X_test), Y_test)


______________________________________________
Classifier: Random Forest Tree
AUC: 0.8616421823460098
______________________________________________

